<a href="https://colab.research.google.com/github/TA-aiacademy/course_3.0/blob/tsrnn/07_TSRNN/TSRNN_Part2/1_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports and Installs

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
from plotly import express as px

import numpy as np
import tensorflow.data as tfd

## Utilities

In [2]:
def plot_series(time, series, start=0, end=None, labels=None, title=None):
    #  Visualizes time series data
    # Args:
    #  time (array of int) - 時間點, 長度為T
    #  series (list of array of int) - 時間點對應的資料列表，列表內時間序列數量為D，
    #                                  每筆資料長度為T，若非為列表則轉為列表
    #  start (int) - 開始的資料序(第幾筆)
    #  end (int) -   結束繪製的資料序(第幾筆)
    #  labels (list of strings)- 對於多時間序列或多維度的標註
    #  title (string)- 圖片標題

    # 若資料只有一筆，則轉為list
    if type(series) != list:
        series = [series]

    if not end:
        end = len(series[0])

    if labels:
        # 設立dictionary, 讓plotly畫訊號線時可以標註label
        dictionary = {"time": time}
        for idx, l in enumerate(labels):
            # 截斷資料，保留想看的部分，並分段紀錄於dictionary中
            dictionary.update({l: series[idx][start:end]})
        # 畫訊號線
        fig = px.line(dictionary,
                      x="time",
                      y=list(dictionary.keys())[1:],
                      width=1000,
                      height=400,
                      title=title)
    else:
        # 畫訊號線
        fig = px.line(x=time, y=series, width=1000, height=400, title=title)
    fig.show()


# 合成資料生成
def trend(time, slope=0):
    # 產生合成水平直線資料，其長度與時間等長，直線趨勢與設定slope相同
    # Args:
    #  time (array of int) - 時間點, 長度為T
    #  slope (float) - 設定資料的傾斜程度與正負
    # Returns:
    #  series (array of float) -  產出slope 與設定相同的一條線

    series = slope * time

    return series


def seasonal_pattern(season_time, pattern_type='triangle'):
    # 產生某個特定pattern，
    # Args:
    #  season_time (array of float) - 周期內的時間點, 長度為T
    #  pattern_type (str) -  這邊提供triangle與cosine
    # Returns:
    #  data_pattern (array of float) -  根據自訂函式產出特定的pattern

    # 用特定function生成pattern
    if pattern_type == 'triangle':
        data_pattern = np.where(season_time < 0.5,
                                season_time*2,
                                2-season_time*2)
    if pattern_type == 'cosine':
        data_pattern = np.cos(season_time*np.pi*2)

    return data_pattern


def seasonality(time, period, amplitude=1, phase=30, pattern_type='triangle'):
    # Repeats the same pattern at each period
    # Args:
    #   time (array of int) - 時間點, 長度為T
    #   period (int) - 週期長度，必小於T
    #   amplitude (float) - 序列幅度大小
    #   phase (int) - 相位，為遞移量，正的向左(提前)、負的向右(延後)
    #   pattern_type (str) -  這邊提供triangle與cosine
    # Returns:
    #   data_pattern (array of float) - 有指定周期、振幅、相位、pattern後的time series

    # 將時間依週期重置為0
    season_time = ((time + phase) % period) / period

    # 產生週期性訊號並乘上幅度
    data_pattern = amplitude * seasonal_pattern(season_time, pattern_type)

    return data_pattern


def noise(time, noise_level=1, seed=None):
    # 合成雜訊，這邊用高斯雜訊，機率密度為常態分布
    # Args:
    #   time (array of int) - 時間點, 長度為T
    #   noise_level (float) - 雜訊大小
    #   seed (int) - 同樣的seed可以重現同樣的雜訊
    # Returns:
    #   noise (array of float) - 雜訊時間序列

    # 做一個基於某個seed的雜訊生成器
    rnd = np.random.RandomState(seed)

    # 生與time同長度的雜訊，並且乘上雜訊大小 (不乘的話，標準差是1)
    noise = rnd.randn(len(time)) * noise_level

    return noise


def toy_generation(time=np.arange(4 * 365),
                   bias=500.,
                   slope=0.1,
                   period=180,
                   amplitude=40.,
                   phase=30,
                   pattern_type='triangle',
                   noise_level=5.,
                   seed=2022):
    signal_series = bias\
                  + trend(time, slope)\
                  + seasonality(time,
                                period,
                                amplitude,
                                phase,
                                pattern_type)
    noise_series = noise(time, noise_level, seed)

    series = signal_series+noise_series
    return series


# Dataset
def win_ar_ds(series, size, shift=1):
    # 輸出Window-wise Forcasting Dataset
    # Args:
    #   series (array of float) - 時序資料, 長度為T
    #   size (int) - Window大小
    #   shift (int) - 每個window起始點間距
    # Returns:
    #   (tf.data.Dataset(母類名稱，切確type為MapDataset)) -
    #   - 一個一次生一個window的生成器

    ds = tfd.Dataset.from_tensor_slices(series)
    ds = ds.window(size=size+1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda ds: ds.batch(size+1))
    return ds.map(lambda x: (x[:-1], x[-1:]))

# 評估function
def MAE(pred, gt):
    # 計算Mean Absolute Error
    # Args:
    #  pred (array of float) - 預測資料
    #  gt (array of float) - 答案資料
    # Returns:
    #  計算結果 (float)
    return abs(pred-gt).mean()


def MSE(pred, gt):
    # 計算Mean Square Error
    # Args:
    #  pred (array of float) - 預測資料
    #  gt (array of float) - 答案資料
    # Returns:
    #  計算結果 (float)
    return pow(pred-gt, 2).mean()


def R2(pred, gt):
    # 計算R square score
    # Args:
    #  pred (array of float) - 預測資料
    #  gt (array of float) - 答案資料
    # Returns:
    #  計算結果 (float)
    return 1-pow(pred-gt, 2).sum()/pow(gt-gt.mean(), 2).sum()

## Generate the Synthetic Data

In [3]:
def split(x, train_size):
    return x[..., :train_size], x[..., train_size:]


# 先合成資料，還有作資料分割
time = np.arange(4*365)  # 定義時間點
series_sample = toy_generation(time, pattern_type='cosine')  # 這就是我們合成出來的資料

time_train, time_test = split(time, 3*365)
series_train, series_test = split(series_sample, 3*365)

# 畫出生成的訓練資料
plot_series(time_train,
            series_train,
            labels=['Training Data'])

In [4]:
window_size = 7

# 用資料predict資料的training set
train_ds = win_ar_ds(series_train, size=window_size)  # 切time series
train_loader = train_ds.cache()\
    .shuffle(1000).batch(32, drop_remainder=True).prefetch(-1)

# 用資料predict資料的testing set
test_ds = win_ar_ds(series_test, size=window_size)  # 切time series
test_loader = test_ds.batch(32).prefetch(-1)

for x, y in train_loader:
    pass

print(x.shape, y.shape)

(32, 7) (32, 1)


## Build the Model

### Many-to-Many or Many-to-One

下面的模型主要由 SimpleRNN layers 組成

模型中堆疊了兩層 RNN，由於第一層 RNN 應該將每個 timestep 的 **output** 傳給第二層 RNN 作為 **input**

因此第一層 RNN 的 `return_sequences` 參數應設為 **True**

而第二層 RNN 會在 input 的最後一個時間點接上 Dense Layer 做出下個時間點的預測，因此 `return_sequences` 參數應設為 **False**

#### `return_sequences` = True:
<img src="https://i.imgur.com/oYuJQrB.png" width = 400>


#### `return_sequences` = False:

<img src="https://i.imgur.com/AkcZMaA.png" width = 400>

### Input Shape

SimpleRNN 的輸入為包含 `[batch, timesteps, feature]` 的 3 維張量輸入

原來的資料窗口需從 (32, 7) reshape 為 (32, 7, 1)。 這表示窗口中的 7 個數據點將映射到 RNN 的 7 個時間步長

* **Reshape 可以在輸入進模型前進行**

* **也可以使用 Lambda 層在模型本身內執行此操作:** <br>
    下面定義了一個 *lambda* 函數，該函數在輸入的最後一個軸上添加一個維度 <br>
    如此可將送進 RNN 的 input_shape 由 `(32, 7)` 改變為 `(32, 7, 1)`

### Model Output
SimpleRNN 默認使用 *tanh* 為激活函數，輸出範圍為 [-1,1]

<img src="https://i.imgur.com/YnmY71z.png" width=400>

而 training data 的值卻落在 400 以上，在模型輸出前可使用另一個 Lambda() 層將輸出進行縮放 x100

In [5]:
import tensorflow as tf
from tensorflow.keras import models, layers, losses, optimizers

In [6]:
# 建立模型
model_tune = tf.keras.models.Sequential([
  tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),
                      input_shape=[window_size]),
  tf.keras.layers.SimpleRNN(40, return_sequences=True),
  tf.keras.layers.SimpleRNN(40),
  tf.keras.layers.Dense(1),
  tf.keras.layers.Lambda(lambda x: x * 100.0)
])

# 給出模型的 summary
model_tune.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda (Lambda)             (None, 7, 1)              0         
                                                                 
 simple_rnn (SimpleRNN)      (None, 7, 40)             1680      
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 40)                3240      
                                                                 
 dense (Dense)               (None, 1)                 41        
                                                                 
 lambda_1 (Lambda)           (None, 1)                 0         
                                                                 
Total params: 4,961
Trainable params: 4,961
Non-trainable params: 0
_________________________________________________________________


## Tune the Learning Rate

在正式的模型訓練之前，可以先對 learning rate 做優化<br>
設置一個 learning rate scheduler 使其隨著每一個 epoch 動態逐步調大<br>
觀察模型訓練過程中 loss 對於相應 learning rate 的改變<br>
之後正式的模型訓練可以設定讓 loss 下降較快的 learning rate 以提高模型收斂效率<br>

此外，loss function 這裡使用 Huber Loss:

$
L_{\delta}(y,f(x))=\left\{\begin{array}{ll}
                 \frac{1}{2}(y-f(x))^2, & \mbox{for $|y-f(x)|\leq\delta$} \\
                 \delta\cdot(|y-f(x)|-\frac{1}{2}\delta), & \mbox{otherwise.} \\
                \end{array} \right.
$                
<img src="https://i.imgur.com/RNjSrnr.png" width=400>

Huber Loss 可以降低對 outlier data 的懲罰程度，也就是說在訓練時參數收斂的方向比較不會受到 outlier 產生的 loss 的影響

In [7]:
# 設置學習率調節器
lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(epoch / 80))

# 設置優化器，這邊選擇 Adam
optimizer = tf.keras.optimizers.Adam()

# 編譯模型並選擇 Huber loss
model_tune.compile(loss=tf.keras.losses.Huber(), optimizer=optimizer)

# 訓練模型
history = model_tune.fit(
    train_loader, 
    epochs=600, 
    callbacks=[lr_schedule])

Epoch 1/600
34/34 [==============================] - 9s 62ms/step - loss: 539.6182 - lr: 1.0000e-08
Epoch 2/600
34/34 [==============================] - 1s 18ms/step - loss: 539.6011 - lr: 1.0292e-08
Epoch 3/600
34/34 [==============================] - 1s 24ms/step - loss: 539.5839 - lr: 1.0593e-08
Epoch 4/600
34/34 [==============================] - 1s 16ms/step - loss: 539.5668 - lr: 1.0902e-08
Epoch 5/600
34/34 [==============================] - 1s 16ms/step - loss: 539.5482 - lr: 1.1220e-08
Epoch 6/600
34/34 [==============================] - 1s 19ms/step - loss: 539.5280 - lr: 1.1548e-08
Epoch 7/600
34/34 [==============================] - 0s 10ms/step - loss: 539.5079 - lr: 1.1885e-08
Epoch 8/600
34/34 [==============================] - 0s 10ms/step - loss: 539.4877 - lr: 1.2232e-08
Epoch 9/600
34/34 [==============================] - 0s 8ms/step - loss: 539.4673 - lr: 1.2589e-08
Epoch 10/600
34/34 [==============================] - 0s 9ms/step - loss: 539.4468 - lr: 1.2957e-08
E

### Plot Loss vs Learning Rate

下面可以看到 loss 在每個階段隨著不同 learning rate 的變化<br>
之後我們可以選擇一個使 loss 下降較快的 learning rate 作為模型訓練的初始 learning rate

In [8]:
# 定義 learning rate array
lrs = 1e-8 * (10 ** (np.arange(600) / 80))


fig = px.line(x=lrs, y=history.history["loss"], log_x=True)
# fig = px.line(x=lrs, y=history.history["loss"])
fig.update_layout(title='Loss vs Learning Rate', xaxis_title='Learning Rate', yaxis_title='Loss')
fig.update_xaxes(tickformat='0.1e')
fig.show()

## Model Training

In [9]:
# 建立模型
model_rnn = tf.keras.models.Sequential([
  tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),
                      input_shape=[window_size]),
  tf.keras.layers.SimpleRNN(40, return_sequences=True),
  tf.keras.layers.SimpleRNN(40),
  tf.keras.layers.Dense(1),
  tf.keras.layers.Lambda(lambda x: x * 100.0)
])

# 設置初始 learning rate
learning_rate = 1.0e-5

# 設置優化器 
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# 編譯模型
model_rnn.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])

# 訓練模型
history = model_rnn.fit(train_loader, 
    epochs=600, 
    callbacks=[
        tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', patience=5),
        tf.keras.callbacks.EarlyStopping(
            monitor='loss',
            patience=20,
            verbose=2)])

Epoch 1/600
34/34 [==============================] - 2s 6ms/step - loss: 645.0530 - mae: 645.5530 - lr: 1.0000e-05
Epoch 2/600
34/34 [==============================] - 0s 6ms/step - loss: 629.2055 - mae: 629.7055 - lr: 1.0000e-05
Epoch 3/600
34/34 [==============================] - 0s 5ms/step - loss: 612.3815 - mae: 612.8815 - lr: 1.0000e-05
Epoch 4/600
34/34 [==============================] - 0s 5ms/step - loss: 594.8000 - mae: 595.3000 - lr: 1.0000e-05
Epoch 5/600
34/34 [==============================] - 0s 6ms/step - loss: 576.9542 - mae: 577.4542 - lr: 1.0000e-05
Epoch 6/600
34/34 [==============================] - 0s 5ms/step - loss: 559.4575 - mae: 559.9575 - lr: 1.0000e-05
Epoch 7/600
34/34 [==============================] - 0s 5ms/step - loss: 542.7781 - mae: 543.2781 - lr: 1.0000e-05
Epoch 8/600
34/34 [==============================] - 0s 5ms/step - loss: 527.1353 - mae: 527.6353 - lr: 1.0000e-05
Epoch 9/600
34/34 [==============================] - 0s 5ms/step - loss: 512.545

In [10]:
model_rnn.evaluate(test_loader)

12/12 [==============================] - 1s 8ms/step - loss: 9.7111 - mae: 10.2011


[9.711051940917969, 10.201111793518066]

## Model Prediction

In [11]:
forcast = model_rnn.predict(test_loader)[:, 0]
ground_truth_for_view = series_test[window_size:]
time_for_view = time_test[window_size:]

plot_series(time_for_view,
            [forcast, ground_truth_for_view],
            labels=['prediction', 'ground truth'])

12/12 [==============================] - 0s 8ms/step


In [12]:
R2(forcast, ground_truth_for_view)

0.8510140026978982